In [ ]:
# --- IMPORTS ---

import os, re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from sklearn.model_selection import train_test_split

print("TF:", tf.__version__)

# --- CONFIG ---
DATA_DIR = "17flowers"      # as in your repo
NUM_CLASSES = 17
IMAGES_PER_CLASS = 80

IMG_SIZE = (224, 224)
BATCH_SIZE = 32

EPOCHS_HEAD = 8
EPOCHS_FINETUNE = 8

LR_HEAD = 1e-3
LR_FINETUNE = 1e-5

SEEDS = [42, 123] # two runs with different seeds


# --- LOAD DATA ---

def extract_index(fname):
    m = re.search(r"image_(\d+)\.jpg$", fname)
    return int(m.group(1)) if m else None

def build_df(data_dir):
    files = sorted([f for f in os.listdir(data_dir) if f.lower().endswith(".jpg")])
    rows = []
    for f in files:
        idx = extract_index(f)
        if idx is None:
            continue
        label = (idx - 1) // IMAGES_PER_CLASS  # 0..16
        if 0 <= label < NUM_CLASSES:
            rows.append((os.path.join(data_dir, f), idx, label))
    df = pd.DataFrame(rows, columns=["filepath", "idx", "label"]).sort_values("idx").reset_index(drop=True)
    return df

df = build_df(DATA_DIR)
print("Images:", len(df), " Classes:", df["label"].nunique())
df.head()

# --- DATA SPLIT ---
def make_split(df, seed):
    train_df, temp_df = train_test_split(
        df, test_size=0.50, random_state=seed, stratify=df["label"]
    )
    val_df, test_df = train_test_split(
        temp_df, test_size=0.50, random_state=seed, stratify=temp_df["label"]
    )
    return train_df.reset_index(drop=True), val_df.reset_index(drop=True), test_df.reset_index(drop=True)

splits = {}
for seed in SEEDS:
    train_df, val_df, test_df = make_split(df, seed)
    splits[seed] = (train_df, val_df, test_df)
    print(f"Seed={seed} -> train={len(train_df)} val={len(val_df)} test={len(test_df)}")


TF: 2.8.0


In [ ]:
# --- AUGMENTATION + PREPROCESSING ---
AUTOTUNE = tf.data.AUTOTUNE
preprocess = tf.keras.applications.vgg19.preprocess_input

augment = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
], name="augment")




2026-01-11 15:01:34.592806: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
def load_image(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, IMG_SIZE)
    img = tf.cast(img, tf.float32)
    return img

def make_dataset(df, training=False):
    paths = df["filepath"].values
    labels = df["label"].values.astype(np.int32)

    ds = tf.data.Dataset.from_tensor_slices((paths, labels))

    if training:
        ds = ds.shuffle(buffer_size=len(df), reshuffle_each_iteration=True)

    def _map_fn(path, label):
        img = load_image(path)
        if training:
            img = augment(img)
        img = preprocess(img)      # IMPORTANT for ImageNet-pretrained VGG19
        return img, label

    ds = ds.map(_map_fn, num_parallel_calls=AUTOTUNE)
    ds = ds.batch(BATCH_SIZE).prefetch(AUTOTUNE)
    return ds


In [ ]:
# sanity check

seed = SEEDS[0]
train_df, val_df, test_df = splits[seed]
train_ds = make_dataset(train_df, training=True)

batch_imgs, batch_labels = next(iter(train_ds))
print(batch_imgs.shape, batch_labels[:10].numpy())


(32, 224, 224, 3) [ 1 13 10  9  5  4  8  9  4 15]


In [10]:
# --- MODEL DEFINITION ---

def build_vgg19_model(num_classes=NUM_CLASSES):
    base = tf.keras.applications.VGG19(
        include_top=False,
        weights="imagenet",
        input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)
    )
    base.trainable = False  # Phase 1: freeze backbone

    inputs = keras.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
    x = base(inputs, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)

    model = keras.Model(inputs, outputs)
    return model, base

model, base = build_vgg19_model()
model.summary()


80150528/80134624 [==============================] - 4s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 vgg19 (Functional)          (None, 7, 7, 512)         20024384  
                                                                 
 global_average_pooling2d (G  (None, 512)              0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 17)                8721      
                                                                 
Total params: 20,033,105
Trainable params: 8,721
Non-trainable

In [11]:
class TestEvalCallback(keras.callbacks.Callback):
    def __init__(self, test_ds):
        super().__init__()
        self.test_ds = test_ds
        self.test_losses = []
        self.test_accs = []

    def on_epoch_end(self, epoch, logs=None):
        results = self.model.evaluate(self.test_ds, verbose=0)
        # results = [loss, accuracy]
        self.test_losses.append(results[0])
        self.test_accs.append(results[1])


In [12]:
def run_training_for_split(seed):
    train_df, val_df, test_df = splits[seed]
    train_ds = make_dataset(train_df, training=True)
    val_ds   = make_dataset(val_df, training=False)
    test_ds  = make_dataset(test_df, training=False)

    model, base = build_vgg19_model()

    test_cb = TestEvalCallback(test_ds)
    early = keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=4, restore_best_weights=True)

    # ---- Phase 1: train head ----
    model.compile(
        optimizer=keras.optimizers.Adam(LR_HEAD),
        loss=keras.losses.SparseCategoricalCrossentropy(),
        metrics=["accuracy"]
    )

    hist1 = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=EPOCHS_HEAD,
        callbacks=[test_cb, early],
        verbose=1
    )

    # ---- Phase 2: fine-tune top layers ----
    # Unfreeze last conv block (you can tune this)
    base.trainable = True
    for layer in base.layers[:-4]:   # keep most frozen, unfreeze last ~4 layers
        layer.trainable = False

    model.compile(
        optimizer=keras.optimizers.Adam(LR_FINETUNE),
        loss=keras.losses.SparseCategoricalCrossentropy(),
        metrics=["accuracy"]
    )

    hist2 = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=EPOCHS_FINETUNE,
        callbacks=[test_cb, early],
        verbose=1
    )

    # Combine histories
    history = {}
    for k in hist1.history.keys():
        history[k] = hist1.history[k] + hist2.history[k]

    # Also attach test history from callback (length = total epochs actually run)
    history["test_loss"] = test_cb.test_losses
    history["test_accuracy"] = test_cb.test_accs

    # Final test evaluation (best weights already restored by EarlyStopping)
    final_test = model.evaluate(test_ds, verbose=0)
    return model, history, final_test


In [ ]:
seed = SEEDS[0]
model, history, final_test = run_training_for_split(seed)
print("Final test [loss, acc]:", final_test)


Epoch 1/8
 5/22 [=====>........................] - ETA: 1:28 - loss: 10.3880 - accuracy: 0.0750

In [ ]:
def plot_curves(history, title_prefix="VGG19"):
    epochs = range(1, len(history["loss"]) + 1)

    # Accuracy
    plt.figure()
    plt.plot(epochs, history["accuracy"], label="train_acc")
    plt.plot(epochs, history["val_accuracy"], label="val_acc")
    plt.plot(epochs, history["test_accuracy"], label="test_acc")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.title(f"{title_prefix} Accuracy")
    plt.legend()
    plt.show()

    # Loss (Cross-Entropy)
    plt.figure()
    plt.plot(epochs, history["loss"], label="train_loss")
    plt.plot(epochs, history["val_loss"], label="val_loss")
    plt.plot(epochs, history["test_loss"], label="test_loss")
    plt.xlabel("Epoch")
    plt.ylabel("Cross-Entropy Loss")
    plt.title(f"{title_prefix} Cross-Entropy Loss")
    plt.legend()
    plt.show()

plot_curves(history, title_prefix=f"VGG19 (seed={seed})")


In [ ]:
results = []

for seed in SEEDS:
    model, history, final_test = run_training_for_split(seed)
    plot_curves(history, title_prefix=f"VGG19 (seed={seed})")
    results.append({"seed": seed, "test_loss": float(final_test[0]), "test_acc": float(final_test[1])})

results_df = pd.DataFrame(results)
results_df

In [ ]:
print("Mean test acc:", results_df["test_acc"].mean())
print("Std  test acc:", results_df["test_acc"].std())